# Faster RCNN

## 1. Introduction
FasterRCNN is a network that does object detection. As explained by its name, its faster than its descendants RCNN and FastRCNN. How fast? Almost realtime fast. 

**The Slower Process, using FastRCNN, would go like this:**
1. Use an algorithm like <mark>Selective Search</mark> on the image to generate interesting boxes/regions 
2. Run the image through a CNN to get a Feature Map
3. For each of the boxes generated from step 1, use the Feature Map and several fully connected layers to output class + Bounding Box coordinates(will expand more later)

**How FasterRCNN works:**
1. Run the image through a CNN to get a Feature Map
2. Run the Activation Map through a separate network, called the <mark>Region Proposal Network(RPN)</mark>, that outputs interesting boxes/regions (RoI)
3. For the interesting boxes/regions from RPN use several fully connected layer to output class + Bounding Box coordinates

The difference here is that FasterRCNN solved the bottleneck of having to run Selective Search for each image as the first step.

## 2. Architecture:


Let's look at the Faster RCNN 's architecture. As you can see, it has a Region Proposal Network (RPN) to detect RoI region and another network that uses proposal region from RPN to classify object and bouding box around that object.

<img src="architecture.png"/>


## 3. Region Proposal Network
### 3.1 Intention (概要・目的)
Region Proposal Network will try to find all proposal RoI in each image. 
If you read Fast RCNN, you can see that they use Selective Search to extract all RoI in image. Selective Search will extract about 2000~ RoI per image. This cause a bottle neck when training or predicting. 

Faster RCNN passes Fast RCNN because it doesn't use Selective Search, it comes from this idea: 
>With FastRCNN we’re already computing an Activation Map in the CNN, why not run the Activation Map through a few more layers to find the interesting regions, and then finish off the forward pass by predicting the classes + bbox coordinates?

And how can we train RPN? Let's go deeper to see how they implement this network. Before we talk about this, we have to define Anchors, which are different sized boxes for helping detect object of various sizes


### 3.2 Anchor
Anchors, which are different sized boxes for helping detect object of various sizes(e.g. humans, vehicles…etc) In the default configurations, there are 3 scales and 3 ratios, which make for 9 total anchors. At each spot in our feature map, we’re going to run these 9 anchors (more on that later).
As you can see in this images, we use a slicing window to slice over the feature maps. At each window, you will assign to center point $k$ anchors (which has different shape and ratio as described above). 

We have two problem here. The first one is whether this center point with specified anchor has object or not (It is RoI or not). The second one is if it has object, can we predict the RoI 's bouding-box? 


<img src="anchor.png"/>


With each anchor, we need to predict its class (2) and its bounding-box (4). So in each window, we will have $2k$ outputs in *cls layer* and $4k$ outputs in *reg layer*. 

### 3.3 Loss function





RPN自身を更新するための誤差を計算し、
ニューラルネットワークの全体の誤差と併せて更新します。論文中の式は次の通り。

<img src="lossfunction.png"/>

RPNの誤差は基本的には２つの要素で成り立っています。
①その領域は物体か背景であるか、②どこに領域があるかの領域の位置（詳細）を計算します。
特に私が勉強するまでイメージを持てなかったのは「そもそも教師信号をどうやってこの式に当てはめるのか」です。そもそも与えてるのバウンディングボックスとクラスだけなのですが・・といった疑問です。

教師信号の決め方ですが、物体の有無はAnchorと物体のIoU（Intersection over Union）が
0.7（論文中）以上の場合、物体が存在することを示します。（positive anchor）
また、IoUが0.3以下を物体が存在しないこと（negative anchor）を示します。

平たく言ってしまえば、ある程度重なっているAnchorに物体が存在するラベルが振られ、
存在しないAnchorに物体が存在しないラベルが振られます。
それ以外は学習に貢献しません。

Anchorは回帰を行います。
Anchorのデータの回帰ですが、次の通り計算を行って、スケールの調整を行っています。
（細かい記号は元論文を見てください）
この値からsmooth L1と呼ばれる誤差関数を用いて最適化します。

<img src="lossfunction_2.png"/>


## 4. Non-Maximum Suppression

After training the RPN network, you will have a problem. 
There are many proposal regions for only one object. 
So you have to keep only a proposal regions that has the highest accuracy when classifying. 
If you want to understand this algorithm, you can read this paper (Japanese):  
http://meideru.com/archives/3538  

<img src="nms1.jpg"/>



## 5. RoI Pooling
RoI Poolingは分類を行う層への入力を固定次元にする役割があります。  
物体検出の領域は非常に可変長です。同じ画像から検出された椅子とりんごが同じサイズではないのと  
同じようなものです。その得られた領域を次の分類のネットワークで処理するため、  
固定次元に縮小する処理を行います。  

処理の概要はこちらのサイトがわかりやすいです。  
https://qiita.com/yu4u/items/5cbe9db166a5d72f9eb8


## 6. Implementation:
There are so many implementations, but I think you should read the below list first. 
Don't try to implement it from beginning. 

* GitHub - endernewton/tf-faster-rcnn: Tensorflow Faster RCNN for Object Detection  
  https://github.com/endernewton/tf-faster-rcnn

## References;
* Detecting Objects in (almost) Real-time: FasterRCNN Explained with Code   
  https://towardsdatascience.com/fasterrcnn-explained-part-1-with-code-599c16568cff
  
* Deepに理解する深層学習による物体検出 by Keras  
  https://qiita.com/GushiSnow/items/8c946208de0d6a4e31e7#faster-r-cnn
  
* 私がわかりにくいと思った「Faster RCNN」のポイントの解説  
  http://nonbiri-tereka.hatenablog.com/entry/2018/03/07/075835
  
* Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks  
  https://arxiv.org/abs/1506.01497